In [56]:
import numpy as np
import pandas as pd
import mysql.connector
from mysql.connector import Error
import yaml

In [57]:
def flatten(iterable, key = "", result = {}):
    """ Recursive Function which systematicallt Flattens nested lists and/or dictionaries"""
    if type(iterable) == dict:
        for i in iterable:
            data_type = type(iterable[i])
            if data_type in [int, str, float]:
                result[key + i] = iterable[i]
            elif data_type == list:
                flatten(iterable[i], key = key + i + "_", result = result)
            elif data_type == dict:
                flatten(iterable[i], key = key + i + "_", result = result)
    elif type(iterable) == list:
        length = len(iterable)
        for i in range(length):
            data_type = type(iterable[i])
            if data_type in [int, str, float]:
                result[key + str(i)] = iterable[i]
            elif data_type == list:
                flatten(iterable[i], key = key + str(i) + "_", result = result)
            elif data_type == dict:
                flatten(iterable[i], key = key + str(i) + "_", result = result)

    else:
        Error("An unidentified type {} has been passed in as an iterable!".format(type(iterable)))
    return result

In [58]:
with open("bbl_json/README.txt", "r") as file:
    strings = file.readlines()
    data = strings[24::]
    data = [i.split(" - ") for i in data]
    data = [i[0:5] + [i[5][:-1].split(" vs ")[0]] + [i[5][:-1].split(" vs ")[1]] for i in data]
    data = pd.DataFrame(
        {i[0]:i[1::] for i in data}, 
        index=["Club", "Game", "Gender", "Id", "Team_1", "Team_2"]
        ).transpose()
    data = data[["Team_1", "Team_2", "Id"]]

data["Date"] = data.index

In [59]:
for i in range(data.shape[0]):
    index = data.iloc[i]["Id"]
    with open("bbl_json/{}.json".format(index), "r") as file:
        match_data = yaml.safe_load(file)
        file.close()
    info = match_data['info']
    dictionary = flatten(info)
    dictionary["game_id"] = index
    for team_num in range(2):
        for player_num in range(11):
            dictionary["players_team{}_player_{}".format(team_num, player_num)] = dictionary["players_{}_{}".format(dictionary["team"], player_num)]
    break

In [60]:
dictionary

{'balls_per_over': 6,
 'city': 'Carrara',
 'dates_0': '2022-01-04',
 'event_name': 'Big Bash League',
 'event_match_number': 38,
 'gender': 'male',
 'match_type': 'T20',
 'officials_match_referees_0': 'Kepler Wessels',
 'officials_reserve_umpires_0': 'Steven Farrell',
 'officials_tv_umpires_0': 'D Taylor',
 'officials_umpires_0': 'GA Abood',
 'officials_umpires_1': 'Troy Penman',
 'outcome_winner': 'Perth Scorchers',
 'outcome_by_runs': 10,
 'overs': 20,
 'player_of_match_0': 'AM Hardie',
 'players_Perth Scorchers_0': 'KR Patterson',
 'players_Perth Scorchers_1': 'CT Bancroft',
 'players_Perth Scorchers_2': 'C Munro',
 'players_Perth Scorchers_3': 'LJ Evans',
 'players_Perth Scorchers_4': 'AJ Turner',
 'players_Perth Scorchers_5': 'AM Hardie',
 'players_Perth Scorchers_6': 'AC Agar',
 'players_Perth Scorchers_7': 'AJ Tye',
 'players_Perth Scorchers_8': 'P Hatzoglou',
 'players_Perth Scorchers_9': 'TS Mills',
 'players_Perth Scorchers_10': 'LR Morris',
 'players_Sydney Sixers_0': 'JR Ph